In [45]:
import pandas as pd 

In [51]:
df = pd.read_csv("convertcsv.csv")

df


,object,epoch_tdb,tp_tdb,e,i_deg,w_deg,node_deg,q_au_1,q_au_2,p_yr,moid_au,ref,object_name,a1_au_d_2,a2_au_d_2,a3_au_d_2,dt_d
0,P/2004 R1 (McNaught),54629,2455248.548,0.682527,4.894556,0.626838,295.985450,0.986192,5.23,5.48,0.027011,20,P/2004 R1 (McNaught),NaN,NaN,NaN,NaN
1,P/2008 S1 (Catalina-McNaught),55101,2454741.329,0.666313,15.100746,203.649023,111.392003,1.190642,5.95,6.74,0.194101,13,P/2008 S1 (Catalina-McNaught),NaN,NaN,NaN,NaN
2,1P/Halley,49400,2446467.395,0.967143,162.262691,111.332485,58.420081,0.585978,35.08,75.32,0.063782,J863/77,1P/Halley,2.700000e-10,1.550000e-10,NaN,NaN
3,2P/Encke,56870,2456618.204,0.848268,11.779995,186.540346,334.569806,0.336092,4.09,3.30,0.173092,74,2P/Encke,1.580000e-10,-5.050000e-12,NaN,NaN
4,3D/Biela,-9480,2390514.115,0.751299,13.216400,221.658800,250.669000,0.879073,6.19,6.65,0.000518,IAUCAT03,3D/Biela,3.900000e-09,-2.540000e-10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,P/2014 U2 (Kowalski),56974,2456937.650,0.602949,7.495593,35.252210,356.180679,1.164298,4.70,5.02,0.183671,10,P/2014 U2 (Kowalski),NaN,NaN,NaN,NaN
156,P/2015 A3 (PANSTARRS),57040,2457076.250,0.848168,172.513745,249.508614,277.082142,1.153857,14.05,20.95,0.204591,1,P/2015 A3 (PANSTARRS),NaN,NaN,NaN,NaN
157,C/2015 D1 (SOHO),57073,2457073.248,0.994268,69.616289,234.953017,95.881385,0.028321,9.85,10.98,0.515916,3,C/2015 D1 (SOHO),1.250000e-06,NaN,NaN,NaN
158,C/2015 F2 (Polonia),57110,2457141.343,0.962399,28.661030,352.058758,262.481614,1.209544,63.13,182.45,0.198704,6,C/2015 F2 (Polonia),NaN,NaN,NaN,NaN


## Preprocessing

In [47]:
cleaned_asteroid_df = df.dropna()

df.dtypes


object          object
epoch_tdb        int64
tp_tdb         float64
e              float64
i_deg          float64
w_deg          float64
node_deg       float64
q_au_1         float64
q_au_2         float64
p_yr           float64
moid_au        float64
ref             object
object_name     object
a1_au_d_2      float64
a2_au_d_2      float64
a3_au_d_2      float64
dt_d           float64
dtype: object

## Model

In [54]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the dataset
df = pd.read_csv("convertcsv.csv")

# Extract relevant orbital parameters into a new DataFrame
relevant_columns = [
    'object', 'epoch_tdb', 'tp_tdb', 'e', 'i_deg', 'w_deg', 
    'node_deg', 'q_au_1', 'q_au_2', 'p_yr', 'moid_au', 'ref'
]
celestial_df = df[relevant_columns]

# Clean the data: drop rows with missing values
celestial_df = celestial_df.dropna()

# Set the number of objects to process
num_objects = 10
if num_objects > len(celestial_df):
    num_objects = len(celestial_df)
celestial_df = celestial_df.iloc[:num_objects]

# Function to convert degrees to radians
def deg_to_rad(deg):
    return deg * (np.pi / 180)

# Kepler's Equation Solver for Eccentric Anomaly
def solve_kepler(M, e, tol=1e-6):
    E = M
    while True:
        delta_E = (M - (E - e * np.sin(E))) / (1 - e * np.cos(E))
        E += delta_E
        if abs(delta_E) < tol:
            break
    return E

# Function to propagate the orbit and get position at time t
def propagate_orbit(e, a, ma, n, t):
    M_t = ma + n * t  # Mean anomaly at time t (radians)
    E_t = solve_kepler(M_t, e)  # Solve for Eccentric Anomaly
    true_anomaly = 2 * np.arctan2(np.sqrt(1 + e) * np.sin(E_t / 2), np.sqrt(1 - e) * np.cos(E_t / 2))
    r = a * (1 - e * np.cos(E_t))  # Orbital radius
    return r, true_anomaly

# Function to calculate 3D position
def celestial_position_3d(e, a, ma, n, omega, Omega, inclination, days_passed):
    t = days_passed / 365.25  # Convert to years
    r, true_anomaly = propagate_orbit(e, a, ma, n, t)
    x_orbital = r * np.cos(true_anomaly)
    y_orbital = r * np.sin(true_anomaly)

    # Rotate the orbital plane by omega, inclination, and Omega
    omega_rad = deg_to_rad(omega)
    Omega_rad = deg_to_rad(Omega)
    inclination_rad = deg_to_rad(inclination)

    x_prime = x_orbital * np.cos(omega_rad) - y_orbital * np.sin(omega_rad)
    y_prime = x_orbital * np.sin(omega_rad) + y_orbital * np.cos(omega_rad)
    z_prime = 0

    x_rot = x_prime
    y_rot = y_prime * np.cos(inclination_rad) - z_prime * np.sin(inclination_rad)
    z_rot = y_prime * np.sin(inclination_rad) + z_prime * np.cos(inclination_rad)

    x_final = x_rot * np.cos(Omega_rad) - y_rot * np.sin(Omega_rad)
    y_final = x_rot * np.sin(Omega_rad) + y_rot * np.cos(Omega_rad)

    return x_final, y_final, z_rot

# Set today’s date as the reference
today = datetime(2024, 10, 6)
julian_day_offset = 2451545  # Julian Day for 2000-01-01

# Calculate today's Julian Date
today_julian = 2451545 + (today - datetime(2000, 1, 1)).days

# Calculate the time difference between today's Julian date and the comet's epoch
time_diff_years = (today_julian - celestial_df['epoch_tdb']) / 365.25
mean_motion_arr = (2 * np.pi) / celestial_df['p_yr']
celestial_df['mean_anomaly_at_today'] = mean_motion_arr * time_diff_years

# Prepare to store the predictions
prediction_data = []

# Process each comet separately
for _, row in celestial_df.iterrows():
    e = row['e']
    q = row['q_au_1']  # Perihelion distance
    a = q / (1 - e)  # Semi-major axis
    ma = row['mean_anomaly_at_today']  # Use today's mean anomaly
    n = 2 * np.pi / row['p_yr']  # Mean motion
    Omega = row['node_deg']  # Longitude of ascending node
    omega = row['w_deg']  # Argument of perihelion
    inclination = row['i_deg']  # Inclination

    # Loop over the next 3000 days for this specific comet
    for day in range(3000):
        # Calculate position for the current day
        x, y, z = celestial_position_3d(e, a, ma, n, omega, Omega, inclination, day)

        # Store the result along with orbital parameters and day
        prediction_data.append({
            'object': row['object'],
            'day': day,
            'e': e,
            'a': a,
            'ma': ma,
            'n': n,
            'omega': omega,
            'Omega': Omega,
            'inclination': inclination,
            'x': x,
            'y': y,
            'z': z
        })

# Convert the predictions to a DataFrame
prediction_df = pd.DataFrame(prediction_data)

# Save the DataFrame to a CSV file
prediction_df.to_csv('celestial_positions_3000_days_per_comet.csv', index=False)

print("Predictions saved to 'celestial_positions_3000_days_per_comet.csv'")


Predictions saved to 'celestial_positions_3000_days_per_comet.csv'
